In [265]:
import pandas as pd
import time
from tqdm import tqdm
import gc
import warnings 
warnings.filterwarnings('ignore')

# 讀取資料

In [266]:
s = time.time()
# 訓練集
train_data = pd.read_feather('../data/train_handle_nan.feather')
# 測試集
test_data = pd.read_feather('../data/test_handle_nan.feather')
print('cost time:',time.time()-s)
# 展示
print(train_data.shape,test_data.shape)
display(train_data.head())
display(test_data.head())

cost time: 1.0765950679779053
(10184090, 53) (500000, 4)


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,20.0,10379793.0,37,1.0,4380.700782,1.0,0.0,0.0,0.0,1.00,...,2.0,3.0,15.0,1.0,99.0,30.0,102382.569589,0.0,5.0,1.0
1,22.0,10124560.0,other,1.0,20298.249036,1.0,0.0,0.0,0.0,1.00,...,1.0,2.0,11.0,1.0,99.0,30.0,123763.112816,0.0,5.0,1.0
2,19.0,10445673.0,37,1.0,558.394631,1.0,0.0,0.0,0.0,1.00,...,1.0,5.0,15.0,1.0,99.0,30.0,79291.240239,1.0,4.0,1.0
3,10.0,10172273.0,37,2.0,5357.428338,1.0,1.0,0.0,0.0,0.38,...,1.0,3.0,27.0,1.0,6.0,30.0,92480.560013,1.0,3.0,1.0
4,12.0,10296216.0,15,1.0,6159.357893,0.0,0.0,1.0,0.0,0.00,...,1.0,2.0,12.0,1.0,8.0,30.0,128396.971578,1.0,5.0,1.0


,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26


# 定義欄位

In [267]:
print(gc.collect())
feature_columns = list(set(train_data.columns.tolist()) - set(['chid','shop_tag']))
categorical_features = ['masts','educd','trdtp','naty','poscd','cuorg','gender_code','age','primary_card']
numerical_features = list(set(feature_columns)-set(categorical_features))
train_df = train_data
sample_submit = test_data
len(feature_columns),len(categorical_features),len(numerical_features)

63


(51, 9, 42)

# 特徵工程

In [268]:
# 從source_df抓對應的chid算一些統計資訊當作特徵併入target_df裡面
def feature_engineer(target_df,source_df):
    for idx in tqdm(target_df.index):
        # 取出這一條row的chid
        chid = target_df.loc[idx,'chid']
        # 從source_df取出所有屬於這個chid的row
        group_by_chid = source_df.loc[source_df.chid==chid]
        # 數值特徵(numerical_features)統計資訊
        for col in numerical_features: 
            target_df.loc[idx,f"mean_of_{col}"] = group_by_chid.loc[:,col].mean()
            target_df.loc[idx,f"max_of_{col}"] = group_by_chid.loc[:,col].max()
            target_df.loc[idx,f"min_of_{col}"] = group_by_chid.loc[:,col].min()
        # 類別特徵(categorical_features)統計資訊
        for col in categorical_features:
            # 1.nunique (這個欄位有多少種不一樣的數值)
            target_df.loc[idx,f"nunique_of_{col}"] = group_by_chid.loc[:,col].nunique()
            # 2.norm_count (這個欄位在每一種數值上出現的次數(count)除以總數(normalize)等於取平均(mean))
            cat_feat = pd.get_dummies(group_by_chid.loc[:,col],columns=col) # one_hot 的 np.array
            cat_feat = pd.DataFrame(cat_feat.mean(axis=0)).T # 轉成DataFrame取平均
            cat_feat.index = [idx] # 處理DataFrame格式
            cat_feat.columns = [ "norm_count_" + col + "_" + str(i) for i in cat_feat.columns] # 處理DataFrame格式
            try:
                target_df = target_df.join(cat_feat) # 合併DataFrame
            except:
                target_df.loc[idx,cat_feat.columns] = cat_feat.values.reshape(-1)
    return target_df

In [269]:
n = 100
sample_submit_feature = feature_engineer(sample_submit.sample(n),train_data)
train_data_feature = feature_engineer(train_data.sample(n),train_data)

100%|██████████| 100/100 [00:06<00:00, 15.76it/s]


In [270]:
sample_submit_new_features = list(set(sample_submit_feature.columns)-set(['chid','top1','top2','top3']))
common_features = list(set(sample_submit_new_features)&set(train_data_feature.columns))
len(common_features),common_features[0],common_features[-1]

(176, 'nunique_of_naty', 'max_of_card_11_txn_amt_pct')

In [271]:
print(sample_submit_feature[common_features].shape)
print(train_data_feature[common_features].shape) 
display(sample_submit_feature)
display(train_data_feature)


(100, 176)
(100, 176)


,chid,top1,top2,top3,mean_of_overseas_offline_amt_pct,max_of_overseas_offline_amt_pct,min_of_overseas_offline_amt_pct,mean_of_card_10_txn_cnt,max_of_card_10_txn_cnt,min_of_card_10_txn_cnt,...,norm_count_cuorg_14.0,norm_count_age_7.0,norm_count_trdtp_14.0,norm_count_cuorg_18.0,norm_count_trdtp_17.0,norm_count_cuorg_6.0,norm_count_trdtp_3.0,norm_count_trdtp_13.0,norm_count_poscd_5.0,norm_count_trdtp_7.0
8479,10062300,48,18,26,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
488145,10464697,26,19,12,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6372,10269599,22,48,21,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497151,10275835,48,13,6,0.0,0.0,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
477956,10349059,13,36,15,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150150,10298025,15,37,21,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412440,10230012,37,48,36,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
102621,10019357,10,2,25,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447111,10153811,18,37,19,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,norm_count_trdtp_10.0,norm_count_trdtp_9.0,norm_count_trdtp_16.0,norm_count_age_8.0,norm_count_cuorg_4.0,norm_count_cuorg_6.0,norm_count_poscd_6.0,norm_count_cuorg_14.0,norm_count_trdtp_27.0,norm_count_educd_1.0
3892435,17.0,10032770.0,37,1.0,5710.209217,1.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1771742,21.0,10383188.0,2,4.0,14207.618412,4.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3051197,15.0,10148380.0,26,8.0,15253.579153,8.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8484828,21.0,10046816.0,37,2.0,5120.805905,2.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7225981,15.0,10323792.0,48,2.0,10921.166563,2.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5811257,22.0,10020758.0,2,3.0,11462.615286,3.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2486808,21.0,10282746.0,15,4.0,13127.778578,4.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9976858,21.0,10261754.0,9,3.0,11045.649349,3.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287942,10.0,10441378.0,2,1.0,3571.095195,1.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [272]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [273]:
train_miss = missing_values_table(train_data_feature)
train_miss.head(30)

Your selected dataframe has 237 columns.
There are 48 columns that have missing values.


,Missing Values,% of Total Values
norm_count_educd_1.0,99,99.0
norm_count_poscd_6.0,99,99.0
norm_count_trdtp_4.0,99,99.0
norm_count_trdtp_1.0,99,99.0
norm_count_trdtp_9.0,99,99.0
norm_count_age_8.0,99,99.0
norm_count_cuorg_4.0,99,99.0
norm_count_cuorg_6.0,99,99.0
norm_count_trdtp_16.0,99,99.0
norm_count_cuorg_24.0,99,99.0


In [274]:
test_miss = missing_values_table(sample_submit_feature)
test_miss.head(30)

Your selected dataframe has 184 columns.
There are 171 columns that have missing values.


,Missing Values,% of Total Values
norm_count_trdtp_7.0,99,99.0
norm_count_poscd_5.0,99,99.0
norm_count_poscd_7.0,99,99.0
norm_count_trdtp_3.0,99,99.0
norm_count_cuorg_6.0,99,99.0
norm_count_cuorg_18.0,99,99.0
norm_count_trdtp_14.0,99,99.0
norm_count_age_7.0,99,99.0
norm_count_cuorg_14.0,98,98.0
norm_count_poscd_8.0,98,98.0


In [275]:
norm_count_col = [i if 'norm' in i else 'None' for i in train_data_feature.columns.tolist()+sample_submit_feature.columns.tolist()]
norm_count_col = set(norm_count_col)-set(['None'])
train_data_feature[set(norm_count_col)&set(train_data_feature)].describe().T

,count,mean,std,min,25%,50%,75%,max
norm_count_masts_2.0,62.0,0.990860,0.071967,0.433333,1.000000,1.000000,1.000000,1.000000
norm_count_trdtp_13.0,3.0,0.972222,0.048113,0.916667,0.958333,1.000000,1.000000,1.000000
norm_count_cuorg_6.0,1.0,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000
norm_count_poscd_6.0,1.0,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000
norm_count_trdtp_12.0,13.0,0.808110,0.320021,0.033333,0.566667,1.000000,1.000000,1.000000
norm_count_educd_5.0,14.0,0.996032,0.014848,0.944444,1.000000,1.000000,1.000000,1.000000
norm_count_gender_code_1.0,41.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
norm_count_gender_code_0.0,59.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
norm_count_educd_2.0,14.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
norm_count_trdtp_4.0,1.0,0.340909,NaN,0.340909,0.340909,0.340909,0.340909,0.340909


In [276]:
import numpy as np
# 類別型特徵norm_count代表 groupby chid 對應某一個類別欄位 在某一種類別上出現的概率
# 如果為1.0 代表這個 chid 在 某一個類別欄位 其中一種類別上出現的概率為100% 反之為0%
# 如果是缺失值代表這個chid的某一項類別欄位沒有出現過該種類 既然沒有出現過 所以概率= 0%
set_value = 0.0
for col in set(norm_count_col)&set(train_data_feature.columns):
    train_data_feature[col].fillna(set_value,inplace=True)
for col in set(norm_count_col)&set(sample_submit_feature.columns): 
    sample_submit_feature[col].fillna(set_value,inplace=True)

In [277]:
train_miss = missing_values_table(train_data_feature)
train_miss.head(30)

Your selected dataframe has 237 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [278]:
test_miss = missing_values_table(sample_submit_feature)
test_miss.head(30)

Your selected dataframe has 184 columns.
There are 126 columns that have missing values.


,Missing Values,% of Total Values
mean_of_overseas_offline_amt_pct,11,11.0
max_of_card_14_txn_cnt,11,11.0
min_of_card_11_txn_amt_pct,11,11.0
max_of_card_11_txn_amt_pct,11,11.0
mean_of_card_11_txn_amt_pct,11,11.0
min_of_card_1_txn_cnt,11,11.0
max_of_card_1_txn_cnt,11,11.0
mean_of_card_1_txn_cnt,11,11.0
min_of_card_8_txn_cnt,11,11.0
max_of_card_8_txn_cnt,11,11.0


In [279]:
print(set([ i.split('_')[0] for i in train_miss.index]))
print(set([ i.split('_')[0] for i in test_miss.index]))

set()
{'mean', 'max', 'min'}


In [280]:
# 數值型特徵欄位補植
# 數值型特徵代表 groupby chid 在某一個特徵上的,最大,最小,平均值 等等統計數值
# 數值型特徵的"大小"是有意義的會影響激活函數,因此如果缺值補最小值或最大值都太過於極端,因此補"中位數" (平均值會受到極端值影響)
for col in train_miss.index:
    train_data_feature[col] = train_data_feature[col].fillna(train_data_feature[col].median())
for col in test_miss.index:
    sample_submit_feature[col] = sample_submit_feature[col].fillna(sample_submit_feature[col].median())

In [281]:
train_miss = missing_values_table(train_data_feature)
train_miss.head(30)

Your selected dataframe has 237 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [282]:
test_miss = missing_values_table(sample_submit_feature)
test_miss.head(30)

Your selected dataframe has 184 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [283]:
print(train_data_feature.shape)
print(sample_submit_feature.shape)

(100, 237)
(100, 184)


In [284]:
print(train_data_feature[common_features+['shop_tag']].shape)

(100, 177)


In [285]:
print(sample_submit_feature[common_features].shape)

(100, 176)


In [286]:
train_data_feature = train_data_feature[common_features+['shop_tag']]
sample_submit_feature = sample_submit_feature[common_features]

In [287]:
print(train_data_feature.shape)
print(sample_submit_feature.shape)
print(train_data_feature.isnull().sum().sum())
print(sample_submit_feature.isnull().sum().sum())

(100, 177)
(100, 176)
0
0


# 保存

In [288]:
train_data_feature.reset_index().to_feather('../data/train_data_feature.feather')
sample_submit_feature.reset_index().to_feather('../data/sample_submit_feature.feather')